<a href="https://colab.research.google.com/github/sbb2002/Portfolio/blob/main/.study/PyTorch/PyTorch_ch4_logistic_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Youtube
> https://www.youtube.com/watch?v=HgPWRqtg254&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv&index=8

* GitHub
> https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-05_logistic_classification.ipynb

# Logistic regression

* Hypothesis
> $ H(X) = \frac{1}{1+e^{-W^{T}X}} $

* Cost
> $ cost(W) = \sum y log(H(x)) + (1-y)log(1-H(x)) $ \
 * If $y \approx H(x)$, cost is near 0. \
 * If $y \ncong H(x) $, cost goes out $\infty$.\

* Weight update via Gradient Descent
  > $ \begin{align*}
  W & := W - \alpha \frac{\partial}{\partial W} cost(W) \\
  & = W - \alpha \nabla_{W} cost(W)
  \end{align*} $ \
 * $ \alpha = $ (Learning rate)

# Practice

Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

Training data

In [ ]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

print(x_train.shape)
print(y_train.shape)


torch.Size([6, 2])
torch.Size([6, 1])


Computing the Hypothesis

In [ ]:
print("e^(1) equals: ", torch.exp(torch.FloatTensor([1])))

e^(1) equals:  tensor([2.7183])


In [ ]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)


In [ ]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

In [ ]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


In [ ]:
print("1/(1+e^(-1)) equals: ", torch.sigmoid(torch.FloatTensor([1])))


1/(1+e^(-1)) equals:  tensor([0.7311])


In [ ]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [ ]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
torch.Size([6, 1])


Computing the cost function

In [ ]:
losses = -(y_train[0] * torch.log(hypothesis[0]) + (1-y_train[0]) * torch.log(1-hypothesis[0]))
print(losses)

tensor([0.6931], grad_fn=<NegBackward>)


In [ ]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [ ]:
# Torch에서는 위의 모든 것을 한줄로 해결가능하다.
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

## Training w/ low-lv. binary cross entropy loss

In [ ]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [ ]:
# model initialize & set opt
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  # cost 계산
  hypothesis = torch.sigmoid(x_train.matmul(W) + b)
  cost = -(y_train * torch.log(hypothesis) + (1-y_train) * torch.log(1-hypothesis)).mean()

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print("Epoch {:4d}/{} Cost: {:.6f}".format(epoch, nb_epochs, cost.item()))



Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


In [ ]:
# evaluation (여기서는 test-set을 주지 않았으므로 train-set 그대로 evaluate함.)
print("Cost: ")
print(hypothesis[:5])

print("Pred. and Real: ")
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])
print(y_train[:5])

Cost: 
tensor([[2.7711e-04],
        [3.1636e-02],
        [3.9014e-02],
        [9.5618e-01],
        [9.9823e-01]], grad_fn=<SliceBackward>)
Pred. and Real: 
tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [ ]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])   # prediction이 real-value와 같으면 T / 아니면 F

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


## Hi-lv. implementation w/ Class

In [ ]:
class BinaryClassifier(nn.Module):
  def __init__(self, ):
    '''
    이 class를 활용하려면 반드시 x_train, y_train을 정의해야 하며, 아래와 같은 shape을 가져야합니다.
      * x_train의 shape=(rows, features)
      * y_train의 shape=(rows, 1 feature)
      ... 라고 써놓으면 될 듯하다.
    '''
    super().__init__()
    self.linear = nn.Linear(x_train.shape[1], y_train.shape[1])   # input cols, output cols
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    return self.sigmoid(self.linear(x))

In [ ]:
model = BinaryClassifier()

In [ ]:
# set opt
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  hypothesis = model(x_train)
  cost = F.binary_cross_entropy(hypothesis, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 10 == 0:
    prediction = hypothesis >= torch.FloatTensor([0.5])
    correct_prediction = prediction.float() == y_train
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    print("Epoch {:4d}/{} Cost: {:.6f} Accuracy: {:2.2f}%".format(epoch, nb_epochs, cost.item(), accuracy * 100))

Epoch    0/1000 Cost: 0.539713 Accuracy: 83.33%
Epoch   10/1000 Cost: 0.614853 Accuracy: 66.67%
Epoch   20/1000 Cost: 0.441875 Accuracy: 66.67%
Epoch   30/1000 Cost: 0.373145 Accuracy: 83.33%
Epoch   40/1000 Cost: 0.316358 Accuracy: 83.33%
Epoch   50/1000 Cost: 0.266094 Accuracy: 83.33%
Epoch   60/1000 Cost: 0.220498 Accuracy: 100.00%
Epoch   70/1000 Cost: 0.182095 Accuracy: 100.00%
Epoch   80/1000 Cost: 0.157299 Accuracy: 100.00%
Epoch   90/1000 Cost: 0.144091 Accuracy: 100.00%
Epoch  100/1000 Cost: 0.134272 Accuracy: 100.00%
Epoch  110/1000 Cost: 0.125769 Accuracy: 100.00%
Epoch  120/1000 Cost: 0.118297 Accuracy: 100.00%
Epoch  130/1000 Cost: 0.111680 Accuracy: 100.00%
Epoch  140/1000 Cost: 0.105779 Accuracy: 100.00%
Epoch  150/1000 Cost: 0.100483 Accuracy: 100.00%
Epoch  160/1000 Cost: 0.095704 Accuracy: 100.00%
Epoch  170/1000 Cost: 0.091369 Accuracy: 100.00%
Epoch  180/1000 Cost: 0.087420 Accuracy: 100.00%
Epoch  190/1000 Cost: 0.083806 Accuracy: 100.00%
Epoch  200/1000 Cost: 0.08

I can predict conviniently another example using 'class'.

In [ ]:
!git clone https://github.com/deeplearningzerotoall/PyTorch

Cloning into 'PyTorch'...
remote: Enumerating objects: 1899, done.
remote: Total 1899 (delta 0), reused 0 (delta 0), pack-reused 1899
Receiving objects: 100% (1899/1899), 80.33 MiB | 40.60 MiB/s, done.
Resolving deltas: 100% (242/242), done.


In [ ]:
!find -name data-03-diabetes.csv

./PyTorch/data-03-diabetes.csv


In [ ]:
import numpy as np

xy = np.loadtxt('./PyTorch/data-03-diabetes.csv', delimiter=',', dtype=np.float32)

# 1~8 col: input, 9 col: label(real_output)
x_data = xy[:, :-1]
y_data = xy[:, [-1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [ ]:
model = BinaryClassifier()

In [ ]:
# set opt
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  hypothesis = model(x_train)
  cost = F.binary_cross_entropy(hypothesis, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 10 == 0:
    prediction = hypothesis >= torch.FloatTensor([0.5])
    correct_prediction = prediction.float() == y_train
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    print("Epoch {:4d}/{} Cost: {:.6f} Accuracy: {:2.2f}%".format(epoch, nb_epochs, cost.item(), accuracy * 100))

Epoch    0/1000 Cost: 0.712866 Accuracy: 40.32%
Epoch   10/1000 Cost: 0.563852 Accuracy: 67.72%
Epoch   20/1000 Cost: 0.532828 Accuracy: 74.18%
Epoch   30/1000 Cost: 0.514798 Accuracy: 75.76%
Epoch   40/1000 Cost: 0.503438 Accuracy: 77.08%
Epoch   50/1000 Cost: 0.495832 Accuracy: 77.60%
Epoch   60/1000 Cost: 0.490505 Accuracy: 77.47%
Epoch   70/1000 Cost: 0.486647 Accuracy: 77.47%
Epoch   80/1000 Cost: 0.483778 Accuracy: 76.55%
Epoch   90/1000 Cost: 0.481599 Accuracy: 76.55%
Epoch  100/1000 Cost: 0.479915 Accuracy: 76.81%
Epoch  110/1000 Cost: 0.478595 Accuracy: 77.21%
Epoch  120/1000 Cost: 0.477546 Accuracy: 77.21%
Epoch  130/1000 Cost: 0.476702 Accuracy: 77.21%
Epoch  140/1000 Cost: 0.476015 Accuracy: 77.21%
Epoch  150/1000 Cost: 0.475451 Accuracy: 77.08%
Epoch  160/1000 Cost: 0.474984 Accuracy: 77.08%
Epoch  170/1000 Cost: 0.474593 Accuracy: 77.08%
Epoch  180/1000 Cost: 0.474263 Accuracy: 77.21%
Epoch  190/1000 Cost: 0.473982 Accuracy: 77.34%
Epoch  200/1000 Cost: 0.473741 Accuracy: